In [2]:
from flair.data import Sentence
from flair.datasets import ColumnCorpus
from flair.models import SequenceTagger
from flair.embeddings import FlairEmbeddings, StackedEmbeddings
from flair.trainers import ModelTrainer

In [3]:
corpus = ColumnCorpus(
  data_folder = 'data/processed/NER/flair',
  column_format = {0:'text', 1:'ner'},
  train_file = 'train_set',
  dev_file = 'eval_set',
  test_file = 'test_set'
)
tag_dict = corpus.make_tag_dictionary('ner')
print(tag_dict)

2021-01-14 15:27:36,785 Reading data from data/processed/NER/flair
2021-01-14 15:27:36,786 Train: data/processed/NER/flair/train_set
2021-01-14 15:27:36,788 Dev: data/processed/NER/flair/eval_set
2021-01-14 15:27:36,789 Test: data/processed/NER/flair/test_set
Dictionary with 13 tags: <unk>, O, S-dish, B-dish, E-dish, I-dish, B-restaurant, E-restaurant, S-occasion, I-restaurant, S-restaurant, <START>, <STOP>


In [4]:
embeddings = StackedEmbeddings([
  FlairEmbeddings('news-forward-fast'),
  FlairEmbeddings('news-backward-fast')
])

2021-01-14 15:27:39,689 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-news-english-forward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmp43j_urw4


100%|██████████| 19689779/19689779 [00:02<00:00, 8647475.21B/s]

2021-01-14 15:27:42,441 copying /tmp/tmp43j_urw4 to cache at /root/.flair/embeddings/lm-news-english-forward-1024-v0.2rc.pt


2021-01-14 15:27:42,467 removing temp file /tmp/tmp43j_urw4
2021-01-14 15:27:57,970 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-news-english-backward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmp7bqszzul


100%|██████████| 19689779/19689779 [00:02<00:00, 9172932.33B/s]

2021-01-14 15:28:00,935 copying /tmp/tmp7bqszzul to cache at /root/.flair/embeddings/lm-news-english-backward-1024-v0.2rc.pt
2021-01-14 15:28:00,964 removing temp file /tmp/tmp7bqszzul


In [5]:
tagger = SequenceTagger(
    hidden_size = 256,
    embeddings = embeddings,
    tag_dictionary = tag_dict,
    tag_type = 'ner',
    use_crf = True
)

trainer = ModelTrainer(tagger, corpus)
trainer.train(
    'base_flair_model',
    learning_rate=0.1,
    mini_batch_size=30,
    max_epochs=50,
    embeddings_storage_mode='gpu',
    train_with_dev=True
)

2021-01-14 15:28:01,600 ----------------------------------------------------------------------------------------------------
2021-01-14 15:28:01,602 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.25, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 1024)
        (decoder): Linear(in_features=1024, out_features=275, bias=True)
      )
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.25, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 1024)
        (decoder): Linear(in_features=1024, out_features=275, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=2048, out_features=2048, bias=True)
  (rnn): LSTM(2048, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_feature

{'dev_loss_history': [],
 'dev_score_history': [],
 'test_score': 0.5739130434782609,
 'train_loss_history': [46.472652900509715,
  16.024178574724896,
  12.995794691690584,
  11.109108669001882,
  9.903271698370212,
  9.216997495511682,
  8.607914912991408,
  8.168136323370584,
  7.905589813139381,
  7.638228090797982,
  7.364120436877739,
  7.2319772185348885,
  6.919126022152785,
  7.068639662207627,
  6.749530675934582,
  6.549267292022705,
  6.771359641377519,
  6.278719599654035,
  6.385319826079578,
  6.126928503920392,
  6.107369725297137,
  5.960941541485671,
  6.017300547623053,
  5.779660602895225,
  5.621623620754335,
  5.745964940001325,
  5.641582582055069,
  5.467760382629022,
  5.49361379553632,
  5.5047109941156895,
  5.342794156656033,
  5.384584531551454,
  5.408868254684821,
  5.241211094507357,
  5.263221333666546,
  5.162131827052047,
  5.081043574868179,
  4.885121316444583,
  4.865051420723519,
  4.9342628048687445,
  4.820779137495087,
  5.138386604262561,
  4.

In [6]:
model = SequenceTagger.load('base_flair_model/final-model.pt')

2021-01-14 15:56:58,603 loading file base_flair_model/final-model.pt


In [7]:
prueba = Sentence("Everybody was very friendly here. My kids loved the beef tacos and I had a chicken salad. I would probably order tacos or enchiladas next time instead. One of my sons didn't like what he got first and they were happy to exchange it for something else.  My kids had so much fun playing at the playground. I'm so glad to have somewhere close by where my kids can eat and play.")
model.predict(prueba)
print(prueba.to_tagged_string())

Everybody was very friendly here . My kids loved the beef <B-dish> tacos <E-dish> and I had a chicken <B-dish> salad <E-dish> . I would probably order tacos <S-dish> or enchiladas next time instead . One of my sons did n't like what he got first and they were happy to exchange it for something else . My kids had so much fun playing at the playground . I 'm so glad to have somewhere close by where my kids can eat and play .


In [8]:
%%time
trainer = ModelTrainer(tagger, corpus)
trainer.train(
    'base_flair_model',
    learning_rate=0.1,
    mini_batch_size=30,
    max_epochs=50,
    embeddings_storage_mode='gpu',
    train_with_dev=True
)

2021-01-14 15:56:58,862 ----------------------------------------------------------------------------------------------------
2021-01-14 15:56:58,863 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.25, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 1024)
        (decoder): Linear(in_features=1024, out_features=275, bias=True)
      )
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.25, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 1024)
        (decoder): Linear(in_features=1024, out_features=275, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=2048, out_features=2048, bias=True)
  (rnn): LSTM(2048, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_feature

In [9]:
!pip install memory_profiler
%load_ext memory_profiler

  Created wheel for memory-profiler: filename=memory_profiler-0.58.0-cp36-none-any.whl size=30181 sha256=17065669795341906126b4e4866de522e3bf99e48a941fb4d2123d6521355efa
  Stored in directory: /root/.cache/pip/wheels/02/e4/0b/aaab481fc5dd2a4ea59e78bc7231bb6aae7635ca7ee79f8ae5
Successfully built memory-profiler


In [10]:
%%memit
trainer = ModelTrainer(tagger, corpus)
trainer.train(
    'base_flair_model',
    learning_rate=0.1,
    mini_batch_size=30,
    max_epochs=50,
    embeddings_storage_mode='gpu',
    train_with_dev=True
)

2021-01-14 16:24:49,846 ----------------------------------------------------------------------------------------------------
2021-01-14 16:24:49,848 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.25, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 1024)
        (decoder): Linear(in_features=1024, out_features=275, bias=True)
      )
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.25, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 1024)
        (decoder): Linear(in_features=1024, out_features=275, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=2048, out_features=2048, bias=True)
  (rnn): LSTM(2048, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_feature

In [11]:
%%timeit
prueba = Sentence("Everybody was very friendly here. My kids loved the beef tacos and I had a chicken salad. I would probably order tacos or enchiladas next time instead. One of my sons didn't like what he got first and they were happy to exchange it for something else.  My kids had so much fun playing at the playground. I'm so glad to have somewhere close by where my kids can eat and play.")
model.predict(prueba)

10 loops, best of 3: 66.6 ms per loop
